In [24]:
from faker import Faker
import random
import mysql.connector
from datetime import datetime,timedelta

In [2]:
faker=Faker()

In [7]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",     
    password="akshita1234",  
    database="fintech"
)

In [8]:
cursor = conn.cursor()

In [9]:
cities = ['Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Pune', 'Chennai']
kyc_statuses = ['Verified', 'Pending', 'Rejected']
user_count = 1000

In [12]:
for _ in range(user_count):
    full_name = faker.name()
    phone = faker.phone_number()[:10]
    email = faker.email()
    kyc_status = random.choices(kyc_statuses, weights=[0.7, 0.2, 0.1])[0]
    signup_date = faker.date_between(start_date='-2y', end_date='today')
    city = random.choice(cities)
    annual_income = random.randint(200000, 2500000)
    dob = faker.date_of_birth(minimum_age=18, maximum_age=60)

    cursor.execute("""
        INSERT INTO users (full_name, phone, email, kyc_status, signup_date, city, annual_income, dob)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (full_name, phone, email, kyc_status, signup_date, city, annual_income, dob))


In [13]:
conn.commit()
cursor.close()
conn.close()

print(f"{user_count} fake users inserted successfully!")

1000 fake users inserted successfully!


In [15]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",     
    password="akshita1234",  
    database="fintech"
)

In [16]:
cursor = conn.cursor()

In [21]:
cursor.execute("SELECT user_id, annual_income, signup_date FROM users")
users = cursor.fetchall()

In [26]:
count=0
for user_id, income, signup_date in users:
    # Wallet balance: ₹0 to ₹1,00,000
    wallet_balance = round(random.uniform(0, 100000), 2)

    # Credit limit: based on income tier
    if income < 500000:
        credit_limit = random.uniform(5000, 15000)
    elif income < 1000000:
        credit_limit = random.uniform(20000, 50000)
    else:
        credit_limit = random.uniform(60000, 100000)

    # Account created after signup (up to 7 days later)
    created_on = signup_date + timedelta(days=random.randint(0, 7))

    # Insert into accounts table
    cursor.execute("""
        INSERT INTO accounts (user_id, wallet_balance, credit_limit, created_on)
        VALUES (%s, %s, %s, %s)
    """, (user_id, round(wallet_balance, 2), round(credit_limit, 2), created_on))

    count += 1

conn.commit()

In [27]:
cursor.execute("SELECT account_id FROM accounts")
account_ids = [row[0] for row in cursor.fetchall()]

In [29]:
merchants = ['Amazon', 'Flipkart', 'Zomato', 'Swiggy', 'Myntra', 'IRCTC', 'Netflix', 'Google', 'PhonePe', 'Cred', 'Paytm']
txn_types = ['Wallet', 'UPI', 'Credit', 'BNPL', 'Refund']
statuses = ['Success', 'Failed', 'Reversed']
num_txs=15000

In [31]:
for _ in range(num_txs):
    account_id = random.choice(account_ids)
    txn_type = random.choices(txn_types, weights=[0.4, 0.3, 0.2, 0.07, 0.03])[0]
    amount = round(random.uniform(10, 10000), 2)
    txn_time = faker.date_time_between(start_date='-12M', end_date='now')
    merchant = random.choice(merchants)
    status = random.choices(statuses, weights=[0.92, 0.06, 0.02])[0]

    cursor.execute("""
        INSERT INTO transactions (account_id, txn_type, amount, txn_time, merchant, status)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (account_id, txn_type, amount, txn_time, merchant, status))

conn.commit()

In [32]:
cursor.execute("SELECT user_id FROM users")
users = [row[0] for row in cursor.fetchall()]

loan_types = ['BNPL', 'InstantCredit', 'Personal']
statuses = ['Active', 'Closed', 'Defaulted']
loan_count = 0

In [34]:
for user_id in users:
    
    if random.random() < 0.6:
        loan_type = random.choices(loan_types, weights=[0.5, 0.3, 0.2])[0]
        principal = round(random.uniform(5000, 200000), 2)
        tenure = random.choice([3, 6, 9, 12, 18, 24])
        interest_rate = round(random.uniform(9.0, 18.0), 2)
        start_date = faker.date_between(start_date='-1y', end_date='-30d')
        status = random.choices(statuses, weights=[0.6, 0.3, 0.1])[0]

        cursor.execute("""
            INSERT INTO loans (user_id, loan_type, principal, start_date, tenure_months, interest_rate, status)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (user_id, loan_type, principal, start_date, tenure, interest_rate, status))

        loan_count += 1

conn.commit()

In [35]:
cursor.execute("SELECT loan_id, principal, tenure_months, start_date FROM loans")
loans = cursor.fetchall()

repayment_count = 0

In [36]:
for loan_id, principal, tenure, start_date in loans:
    monthly_amount = round(principal / tenure, 2)

    for month in range(tenure):
        due_date = start_date + timedelta(days=30 * month)

        
        roll = random.random()
        if roll < 0.75:
            status = 'Paid'
            delay = random.choice([0, 1, 2, 3])  
        elif roll < 0.90:
            status = 'Late'
            delay = random.randint(4, 15)
        else:
            status = 'Missed'
            delay = None

        paid_on = None if status == 'Missed' else due_date + timedelta(days=delay)

        cursor.execute("""
            INSERT INTO repayments (loan_id, due_date, paid_on, amount_paid, status)
            VALUES (%s, %s, %s, %s, %s)
        """, (loan_id, due_date, paid_on, monthly_amount if status != 'Missed' else 0.00, status))

        repayment_count += 1

conn.commit()

In [37]:
cursor.execute("SELECT user_id FROM users")
user_ids = [row[0] for row in cursor.fetchall()]

sources = ['CIBIL', 'Experian', 'InternalAI']
score_count = 0

In [38]:
for user_id in user_ids:
    for month_offset in range(12):
        score_date = datetime.today().replace(day=1) - timedelta(days=month_offset*30)
        score = random.randint(300, 900)  # CIBIL scale
        source = random.choice(sources)

        cursor.execute("""
            INSERT INTO credit_scores (user_id, score, reported_on, source)
            VALUES (%s, %s, %s, %s)
        """, (user_id, score, score_date.date(), source))

        score_count += 1

conn.commit()

In [39]:
cursor.execute("SELECT txn_id, txn_time FROM transactions WHERE amount > 7500 AND status = 'Success'")
txns = cursor.fetchall()

In [42]:
flag_reasons = [
    "High-value transaction",
    "Unusual merchant pattern",
    "Multiple failed attempts",
    "Transaction at odd hours",
    "IP/location mismatch"
]

In [43]:
flag_count=0
for txn_id, txn_time in txns:
    if random.random() < 0.25:  # 25% of high-value txns are flagged
        reason = random.choice(flag_reasons)
        flagged_on = txn_time + timedelta(minutes=random.randint(1, 60))
        resolved = random.choice([True, False])

        cursor.execute("""
            INSERT INTO fraud_flags (txn_id, reason, flagged_on, resolved)
            VALUES (%s, %s, %s, %s)
        """, (txn_id, reason, flagged_on, resolved))

        flag_count += 1

conn.commit()